# Clustering

In [27]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

import env

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Data Wrangling Exercises

### Acquire(acquire.py)

#### Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

acquire & summarize
Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
Only include properties that include a latitude and longitude value.

In [94]:
# url = env.get_db_url('zillow')

# zillow = pd.read_sql('''
# SELECT *
# FROM properties_2017
# JOIN (SELECT parcelid as pid, MAX(transactiondate) as maxdate FROM predictions_2017 GROUP BY parcelid) as last_date
# ON last_date.pid = parcelid
# LEFT JOIN (SELECT parcelid as pid, transactiondate as maxdate, logerror FROM predictions_2017) as log
# ON last_date.pid = log.pid AND last_date.maxdate = log.maxdate
# LEFT JOIN propertylandusetype
# USING(propertylandusetypeid)
# LEFT JOIN storytype
# USING(storytypeid)
# LEFT JOIN typeconstructiontype
# USING(typeconstructiontypeid)
# LEFT JOIN airconditioningtype
# USING(airconditioningtypeid)
# LEFT JOIN architecturalstyletype
# USING(architecturalstyletypeid)
# LEFT JOIN buildingclasstype
# USING(buildingclasstypeid)
# LEFT JOIN heatingorsystemtype
# USING(heatingorsystemtypeid)
# ''',url)

# zillow.to_csv('zillow.csv',index=False)

In [91]:
# zillow = pd.read_csv('zillow.csv')

In [95]:
# zillow = zillow.drop(columns=['id','pid','maxdate'])

In [99]:
# zillow = zillow.dropna(subset=['latitude','longitude'])

Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [100]:
# zillow.shape

(77381, 66)

In [216]:
# zillow.describe(include='all')

In [270]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 0 to 77413
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   heatingorsystemtypeid         49440 non-null  float64
 1   buildingclasstypeid           15 non-null     float64
 2   architecturalstyletypeid      206 non-null    float64
 3   airconditioningtypeid         24953 non-null  float64
 4   typeconstructiontypeid        222 non-null    float64
 5   storytypeid                   50 non-null     float64
 6   propertylandusetypeid         77381 non-null  float64
 7   parcelid                      77381 non-null  int64  
 8   basementsqft                  50 non-null     float64
 9   bathroomcnt                   77381 non-null  float64
 10  bedroomcnt                    77381 non-null  float64
 11  buildingqualitytypeid         49672 non-null  float64
 12  calculatedbathnbr             76772 non-null  float64
 13  d

In [214]:
# zillow.hist(figsize=(40,20), bins =20)
# plt.show()

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [212]:
def missing_rows(df):
    return pd.concat([
           df.isna().sum().rename('count'),
           df.isna().mean().rename('percent')
           ], axis=1)

# missing_rows(zillow)

Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [213]:
def missing_columns(df):
    col_missing = pd.concat([
    df.isna().sum(axis=1).rename('num_cols_missing'),
    df.isna().mean(axis=1).rename('pct_cols_missing'),
    ], axis=1).value_counts().sort_index()
    col_missing = pd.DataFrame(col_missing)
    col_missing.rename(columns={0:'num_rows'},inplace=True)
    return col_missing.reset_index()

# missing_columns(zillow)

## Prepare
Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [243]:
prop_type_single = zillow[zillow['propertylandusetypeid'].isin([260,261,263,264,266,267,269])]

In [266]:
prop_type_single.unitcnt.drop(labels=[2,3,4,6],inplace = True)

In [267]:
prop_type_single.unitcnt.value_counts(dropna=False)

1.0    47263
NaN    26402
2.0       33
3.0        3
4.0        2
6.0        1
Name: unitcnt, dtype: int64

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.

Decide how to handle the remaining missing values:

- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

## wrangle_zillow.py

Functions of the work above needed to acquire and prepare a new sample of data.

# Mall Customers

## notebook

- Acquire data from mall_customers.customers in mysql database.
- Summarize data (include distributions and descriptive statistics).
- Detect outliers using IQR.
- Split data (train, validate, and test split).
- Encode categorical columns using a one hot encoder (pd.get_dummies).
- Handles missing values.
- Scaling

## wrangle_mall.py

- Acquire data from mall_customers.customers in mysql database.
- Split the data into train, validate, and split
- One-hot-encoding (pd.get_dummies)
- Missing values
- Scaling

Add, commit, and push all of your work.